In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

base_url = 'http://naeb.brit.org/uses/species/?page={}'

plants_list = []

for page_num in range(1, 23):
    url = base_url.format(page_num)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    for link in soup.find_all('a'):
        href = link.get('href')
        if href and not href.startswith('#'):
            match = re.search(r'/uses/species/(\d+)/$', href)
            if match:
                plants_list.append({'text': link.text.strip(), 'link': href})

plants_df = pd.DataFrame(plants_list)

plants_df


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

base_url = 'http://naeb.brit.org/uses/species/?page={}'

plants_list = []

for page_num in range(1, 23):
    url = base_url.format(page_num)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    for link in soup.find_all('a'):
        href = link.get('href')
        if href and not href.startswith('#'):
            match = re.search(r'/uses/species/(\d+)/$', href)
            if match:
                plants_list.append({'text': link.text.strip(), 'link': href})

plants_df = pd.DataFrame(plants_list)

plants_df

# Create additional dataframes
df_list = []

for idx, row in plants_df.iterrows():
    url = row['link']
    page = requests.get('http://naeb.brit.org' + row['link'])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', {'class': 'table table-bordered'})

    if table:
        df = pd.read_html(str(table))[0]
        df_list.append(df)
        df.to_csv(f"{row['text']}.csv")

# Print the resulting dataframes
for df in df_list:
    print(df)


In [6]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

base_url = 'http://naeb.brit.org/uses/species/?page={}'

plants_list = []

for page_num in range(1, 23):
    url = base_url.format(page_num)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    for link in soup.find_all('a'):
        href = link.get('href')
        if href and not href.startswith('#'):
            match = re.search(r'/uses/species/(\d+)/$', href)
            if match:
                plants_list.append({'text': link.text.strip(), 'link': href})

plants_df = pd.DataFrame(plants_list)

# Create a dictionary to store the dataframes
dfs_dict = {}

for index, row in plants_df.iterrows():
    plant_name = row['text']
    url = 'http://naeb.brit.org' + row['link']
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Extract data and create a new dataframe
    data = []
    for tr in soup.find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) == 2:
            data.append({'part': tds[0].text.strip(),
                        'use': tds[1].text.strip()})
    df = pd.DataFrame(data)

    # Add the dataframe to the dictionary with the plant name as the key
    dfs_dict[plant_name] = df

# Print the initial dataframe and the dictionary of dataframes
print(plants_df)
print(dfs_dict)
